# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

%matplotlib inline



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [34]:
df = pd.read_csv('data.csv', index_col = 0)
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tambura,5.6000,27.4667,71.22,41,75,4.03,SS,1639971071
1,Hithadhoo,-0.6000,73.0833,82.15,76,90,7.18,MV,1639971071
2,Zaysan,47.4667,84.8667,26.10,88,100,2.48,KZ,1639971072
3,Kahului,20.8947,-156.4700,74.46,92,75,16.11,US,1639971072
4,Kutum,14.2000,24.6667,54.82,32,0,6.31,SD,1639971072
...,...,...,...,...,...,...,...,...,...
540,Morón,-34.6534,-58.6198,76.98,77,0,3.78,AR,1639971623
541,Koumac,-20.5667,164.2833,82.47,69,58,5.73,NC,1639971886
542,Muhos,64.8080,25.9931,4.78,91,7,4.74,FI,1639971886
543,Tulun,54.5611,100.5792,-10.93,94,64,2.98,RU,1639971886


## Create heatmap

In [35]:
#configure gmaps
gmaps.configure(api_key=g_key)


In [36]:
# Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = df["Humidity"].astype(float)

In [37]:
# Plot Heatmap
fig = gmaps.figure(center=(0, 0), zoom_level= 2)
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=225,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria


In [38]:
#create a list of ideal locations based on weather attributes.
ideal_df = df.loc[(df["Wind Speed"] <= 15) & (df["Cloudiness"] < 10) & 
                  (df["Humidity"] >= 30) & (df["Humidity"] <= 70) &
                  (df["Max Temp"] >= 75) & (df["Max Temp"] <= 85)].dropna()


hotel_df = ideal_df.loc[:,["City","Country","Lat","Lng"]]


In [39]:
#add a column for Hotel Names

hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
417,Makakilo City,US,21.3469,-158.0858,
432,Ratnagiri,IN,16.9833,73.3000,
470,Lakes Entrance,AU,-37.8811,147.9810,
519,Pathein,MM,16.7833,94.7333,
531,Calabar,NG,4.9517,8.3220,


In [40]:
#create a base URL and set parameters for the the API lookup 
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key,
         }

#iterate through the df to find name, lat and long of hotel closest to the city. 
for index, row in hotel_df.iterrows():

    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    # update location key value with lat and lng for current city
    params['location'] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    

    #print closest hotel and add lat, lng to new variable and update Hotel Name column.
    try:
        print(f"The closest hotel to {city} is {hotels['results'][0]['name']}.")
        hotel_lat = hotels['results'][0]['geometry']['location']['lat']
        hotel_lng = hotels['results'][0]['geometry']['location']['lng']
        hotel_df.loc[index, "Hotel Name"] =  hotels['results'][0]['name']
 
    #create an exception for key and index errors and print explanation, update Hotel Name indicating none found
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, "Hotel Name"] = "None Found"
        
        
    print("\n")



The closest hotel to Makakilo City is Four Seasons Resort Oahu at Ko Olina.


The closest hotel to Ratnagiri is Hotel Mathura Executive.


The closest hotel to Lakes Entrance is The Esplanade Resort & Spa.


The closest hotel to Pathein is The First Hotel.


The closest hotel to Calabar is Transcorp Hotels Calabar.




In [41]:
#Verify dataframe has been updated with Hotel Name.
hotel_df

,City,Country,Lat,Lng,Hotel Name
417,Makakilo City,US,21.3469,-158.0858,Four Seasons Resort Oahu at Ko Olina
432,Ratnagiri,IN,16.9833,73.3000,Hotel Mathura Executive
470,Lakes Entrance,AU,-37.8811,147.9810,The Esplanade Resort & Spa
519,Pathein,MM,16.7833,94.7333,The First Hotel
531,Calabar,NG,4.9517,8.3220,Transcorp Hotels Calabar


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [43]:

#Set the coordinates to be marked
coordinates = hotel_lat, hotel_lng


# Plot a map
fig = gmaps.figure(center=(0, 0), zoom_level= 2)
# Create marker layer with info box
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add heat and marker layers
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))